# Install the snippet notebook

Store the snippet notebook on your Google Drive. Then copy the updated URL to 
Tools --> Settings --> Site --> Custom snippet notebook URL. Then when you open 
a new notebook, the snippets will be available.

1.   File --> Save Copy in Drive
2.   Tools --> Settings --> Site --> Custom snippet notebook URL


# Install cctbx (1/2)

This prodedure is adapted from Billy Poon's example: 
https://github.com/phenix-project/Colabs/blob/main/Start_cctbx.ipynb

The installation of cctbx involves a two step process:

1.   Install condacolab
2.   Install cctbx with mamba and move certain libraries to better locations.

The module condacolab install conda and mamba. Mamba is faster.
A progress bar will be updated as the snippet runs.

Completion of the first step will involve the generation of the error message:

**Your session crashed for an unknown reason.** 

Click on the X to close the error message 
before running **Install cctbx (2/2)**


In [ ]:
from IPython.utils import io
import tqdm.notebook

total = 50
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:
        !pip install -q condacolab
        pbar.update(10)

        import condacolab
        condacolab.install()
        pbar.update(40)   

# Install cctbx (2/2)

Use mamba to install cctbx-base.

In [ ]:

with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:
        !mamba install -q cctbx-base
        pbar.update(10)

        # conda installs ${CONDA_PREFIX}/share/cctbx into /usr/local instead of /usr
        !cp -af /usr/local/share/cctbx /usr/share/

        # Add libraries to the sys.path
        import sys
        for d in ['/usr/local/lib/python3.7/lib-dynload', '/usr/local/lib']:
          if d not in sys.path:
            sys.path.insert(0, d)
        pbar.update(10)

        # Check installation
        import os
        if os.path.isdir('/usr/local/share/cctbx') \
          and '/usr/local/lib/python3.7/lib-dynload' in sys.path \
          and '/usr/local/lib' in sys.path:
          print('Finished installing cctbx-base')
        else:
          raise RuntimeError('There was an error fixing up the installation of cctbx-base')
        pbar.update(30)

# Test cctbx

- The code prints three sets of parameter values below the code block.


In [ ]:
from cctbx import crystal
from cctbx import miller

ms = miller.build_set(
    crystal_symmetry=crystal.symmetry(
        space_group_symbol="P4",
        unit_cell=(150.8,150.8,250.4,90.0,90.0,90.0)),
    anomalous_flag=False,
    d_min=3.4)
msu = ms.map_to_asu()
[print(hkl) for hkl in msu.indices()]
print(msu.show_comprehensive_summary())

# Copy mtz files from Google Drive

Load mtz file(s) that you want to analyze onto Google Drive in a created-folder called `dataFiles` and then mount the Google Drive.



In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%shell cp ./drive/MyDrive/dataFiles/*.mtz .

# iotbx extractCrystalSymmetry

- Extract crystal symmetry from mtz file.
- To fetch the X-ray data in a cif file and convert to a mtz file, run the following command on your local computer after installing phenix.

```bash
phenix.fetch_pdb --mtz 3nd4
```

- The `$` below marks a site for editing.

```python
from __future__ import absolute_import, division, print_function
from iotbx import mtz
from cctbx import crystal

def extract_from(file_name):
  mtz_object = mtz.object(file_name=file_name)
  assert mtz_object.n_symmetry_matrices() > 0
  return mtz_object.crystals()[0].crystal_symmetry()
  
 extract_from(file_name="${1:3nd4}.mtz")
 ```



In [ ]:
from __future__ import absolute_import, division, print_function
from iotbx import mtz
from cctbx import crystal

def extract_from(file_name):
  mtz_object = mtz.object(file_name=file_name)
  assert mtz_object.n_symmetry_matrices() > 0
  return mtz_object.crystals()[0].crystal_symmetry()
  
extract_from(file_name="3nd4.mtz")


# iotbx fetchAtomcCif

- Fetch  atomic coordinates from RCSB in mmCIF format.
- The `$` below marks a site for editing.

```python
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="${1:3nd4}", data_type="xray", mirror="rcsb", format="cif", log=sys.stdout)
```

In [ ]:
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="3nd4",data_type="xray", mirror="rcsb", format="cif", log=sys.stdout)


## iotbx fetchFASTA

- Fetch fasta file from RCSB.

- The `$` below marks a site for editing.

```python
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="3nd4",data_type="pdb", mirror="rcsb", format="pdb", log=sys.stdout)
```

In [ ]:
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="3nd4",data_type="pdb", mirror="rcsb", format="pdb", log=sys.stdout)


# iotbx fetchPDB

- Fetch pdb file from RCSB in PDB format.

- The `$` below marks a site for editing.

```python
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="${1:3nd4}",data_type="pdb", mirror="rcsb", format="pdb", log=sys.stdout)
```

In [ ]:
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="3nd4",data_type="pdb", mirror="rcsb", format="pdb", log=sys.stdout)


# iotbx fetchXrayCif

- Fetch X-ray data from RCSB in mmCIF format.
- The `$` below marks a site for editing.

```python
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="${1:3nd4}", data_type="xray", mirror="rcsb", format="cif", log=sys.stdout)
```

In [ ]:
from iotbx.pdb.fetch import get_pdb
import sys
get_pdb(id="3nd4", data_type="xray", mirror="rcsb", format="cif", log=sys.stdout)


# iotbx symmetryFromPDB

- Print the symmetry from a PDB file.
- The `$` below marks a site for editing.

```python
import iotbx
iotbx.pdb.crystal_symmetry_from_pdb.extract_from("${1:3nd4}.pdb")
```


In [ ]:
import iotbx
iotbx.pdb.crystal_symmetry_from_pdb.extract_from("3nd4.pdb")

# millerArrays changeMtzColumns

- Read in mtz file from current directory and write out with fewer columns.
- The `$` below marks a site for editing.

```python
from iotbx.reflection_file_reader import any_reflection_file
hkl_in = any_reflection_file("${1:/Users/blaine/manuscripts/RETkinaseLoxo/ret_blu.mtz}")

miller_arrays = hkl_in.as_miller_arrays()

i_obs =  miller_arrays[3]
r_free_flags = miller_arrays[0]
f_obs = i_obs.f_sq_as_f()

mtz_dataset = i_obs.as_mtz_dataset(column_root_label="I")
mtz_dataset.add_miller_array(f_obs, column_root_label="F")
mtz_dataset.add_miller_array(r_free_flags,column_root_label="${2:FreeR_flag}")
mtz_dataset.mtz_object().write("${3:loxodata.mtz}")
```

In [ ]:
from iotbx.reflection_file_reader import any_reflection_file
hkl_in = any_reflection_file("ret_blu.mtz")

miller_arrays = hkl_in.as_miller_arrays()

i_obs =  miller_arrays[3]
r_free_flags = miller_arrays[0]
f_obs = i_obs.f_sq_as_f()

mtz_dataset = i_obs.as_mtz_dataset(column_root_label="I")
mtz_dataset.add_miller_array(f_obs, column_root_label="F")
mtz_dataset.add_miller_array(r_free_flags,column_root_label="FreeR_flag")
mtz_dataset.mtz_object().write("loxodata.mtz")

# millerArrays CNS2mtz

- Convert CNS reflection file into an mtz file.
- The `$` below marks a site for editing.

```python
from iotbx import reflection_file_reader
import os
reflection_file = reflection_file_reader.any_reflection_file(file_name=os.path.expandvars("${1:\$CNS_SOLVE/doc/html/tutorial/data/pen/scale.hkl}"))
from cctbx import crystal
crystal_symmetry = crystal.symmetry( unit_cell=(${2:97.37, 46.64, 65.47, 90, 115.4, 90}), space_group_symbol="${3:C2}")
miller_arrays = reflection_file.as_miller_arrays( crystal_symmetry=crystal_symmetry)
mtz_dataset = None
for miller_array in miller_arrays:
    if (mtz_dataset is None):
        mtz_dataset = miller_array.as_mtz_dataset(
            column_root_label=miller_array.info().labels[0]) 
    else:
        mtz_dataset.add_miller_array(
            miller_array=miller_array, 
            column_root_label=miller_array.info().labels[0])
mtz_object = mtz_dataset.mtz_object() 
mtz_object.show_summary()
```

In [ ]:
from iotbx import reflection_file_reader
import os
reflection_file = reflection_file_reader.any_reflection_file(file_name=os.path.expandvars("\$CNS_SOLVE/doc/html/tutorial/data/pen/scale.hkl"))
from cctbx import crystal
crystal_symmetry = crystal.symmetry( unit_cell=(97.37, 46.64, 65.47, 90, 115.4, 90), space_group_symbol="C2")
miller_arrays = reflection_file.as_miller_arrays( crystal_symmetry=crystal_symmetry)
mtz_dataset = None
for miller_array in miller_arrays:
    if (mtz_dataset is None):
        mtz_dataset = miller_array.as_mtz_dataset(
            column_root_label=miller_array.info().labels[0]) 
    else:
        mtz_dataset.add_miller_array(
            miller_array=miller_array, 
            column_root_label=miller_array.info().labels[0])
mtz_object = mtz_dataset.mtz_object() 
mtz_object.show_summary()

# millerArrays computeAllPossibleMillerIndices

- Compute all possible Miller indices.
- The `$` below marks a site for editing.

```python
from cctbx import miller

def generate_reflection_indices(uc, dmin):
    maxh, maxk, maxl = uc.max_miller_indices(dmin)
    indices = []
    for h in range(-maxh, maxh + 1):
        for k in range(-maxk, maxk + 1):
            for l in range(-maxl, maxl + 1):
                if h == 0 and k == 0 and l == 0:
                    continue
                if uc.d((h, k, l)) < dmin:
                    continue
        indices.append((h, k, l))
    return indices
    
uc=(${1:5.4307,5.4307,5.4307,90.00,90.0,90.00})
dmin=${2:1.0}
```

In [ ]:
from cctbx import miller

def generate_reflection_indices(uc, dmin):
    maxh, maxk, maxl = uc.max_miller_indices(dmin)

    indices = []
    for h in range(-maxh, maxh + 1):
        for k in range(-maxk, maxk + 1):
            for l in range(-maxl, maxl + 1):
                if h == 0 and k == 0 and l == 0:
                    continue
                if uc.d((h, k, l)) < dmin:
                    continue
        indices.append((h, k, l))
    return indices
    
uc=(5.4307,5.4307,5.4307,90.00,90.0,90.00)
dmin=1.0


# millerArrays computeMillerIndicesInASU 

- Compute all possible Miller indices in the asymmetric (ASU).

- The `$`'s below marks a site for editing.

```bash
from cctbx import miller
import cctbx
from cctbx import crystal

ms = miller.build_set(
    crystal_symmetry=crystal.symmetry(
        space_group_symbol="${1:I432}",
        unit_cell=("2:{54.4,54.4,54.4,90.00,90.0,90.00}") ),
    anomalous_flag=${3:False},
    d_min=${4:0.4})

# map the reflections to the asu and print
  
msu = ms.map_to_asu()
[print(hkl2) for hkl2 in msu.indices()]
```

In [ ]:
from cctbx import miller
import cctbx
from cctbx import crystal

ms = miller.build_set(
    crystal_symmetry=crystal.symmetry(
        space_group_symbol="I432",
        unit_cell=("54.4,54.4,54.4,90.00,90.0,90.00") ),
    anomalous_flag=False,
    d_min=5.0)

for hkl in ms.indices():
    print(hkl)

# map the reflections to the asu and print
msu = ms.map_to_asu()
[print(hkl2) for hkl2 in msu.indices()]

# millerArrays computeMillerIndicesUnitCell

- Build miller indices given unit cell and resolution limit.

- The `$` below marks a site for editing.

In [ ]:
from cctbx import crystal
from cctbx import miller

ms = miller.build_set(
    crystal_symmetry=crystal.symmetry(
        space_group_symbol="P4",
        unit_cell=(150.8,150.8,250.4,90.0,90.0,90.0)),
    anomalous_flag=False,
    d_min=1.4)
msu = ms.map_to_asu()
[print(hkl) for hkl in msu.indices()]
print(msu.show_comprehensive_summary())

# millerArrays extractReflectionsInShell

- Extract the reflections in a shell.
- The `$` below marks a site for editing.

```python
from iotbx import mtz
mtz_obj = mtz.object(file_name="${1:3nd4}.mtz")
miller_arrays = mtz_obj.as_miller_arrays()
for miller_array in miller_arrays:
    miller_array_truncated = miller_array.resolution_filter(d_min=${2:2}, d_max=${3:5})
print(miller_array_truncated)
 ```

In [ ]:
from iotbx import mtz
mtz_obj = mtz.object(file_name="3nd4.mtz")
miller_arrays = mtz_obj.as_miller_arrays()
for miller_array in miller_arrays:
    miller_array_truncated = miller_array.resolution_filter(d_min=2, d_max=5)
print(miller_array_truncated)


# millerArrays computeAllMillerIndicesUnitCell

- Build miller indices given unit cell and resolution limit.
- The `$` below marks a site for editing.

```python
from cctbx import crystal
from cctbx import miller

ms = miller.build_set(
    crystal_symmetry=crystal.symmetry(
        space_group_symbol="${1:P4}",
        unit_cell=(${2:150.8,150.8,250.4,90.0,90.0,90.0})),
    anomalous_flag=${3:False},
    d_min=${4:1.4})
msu = ms.map_to_asu()
[print(hkl) for hkl in msu.indices()]
print(msu.show_comprehensive_summary())
```

In [ ]:
from cctbx import crystal
from cctbx import miller

ms = miller.build_set(
    crystal_symmetry=crystal.symmetry(
        space_group_symbol="P4",
        unit_cell=(150.8,150.8,250.4,90.0,90.0,90.0)),
    anomalous_flag=False,
    d_min=1.4)
msu = ms.map_to_asu()
[print(hkl) for hkl in msu.indices()]
print(msu.show_comprehensive_summary())

# millerArrays printColumnLabelsMillerArray

```python
[print(f"Column label: '${1:key[2]}'")  for key in miller_arrays_dict.keys()]
```

In [1]:
[print(f"Column label: key[2]")  for key in miller_arrays_dict.keys()]

NameError: ignored

# millerArrays generateAllMillerIndices

- Compute all possible Miller indices.
- The `$` below marks a site for editing.

```python
from cctbx import miller

def generate_reflection_indices(uc, dmin):
    maxh, maxk, maxl = uc.max_miller_indices(dmin)

    indices = []

    for h in range(-maxh, maxh + 1):
        for k in range(-maxk, maxk + 1):
            for l in range(-maxl, maxl + 1):
                if h == 0 and k == 0 and l == 0:
                    continue
                if uc.d((h, k, l)) < dmin:
                    continue
        indices.append((h, k, l))
    return indices
    
uc=(${1:5.4307,5.4307,5.4307,90.00,90.0,90.00})
dmin=${2:1.0}
```


In [ ]:
from cctbx import miller

def generate_reflection_indices(uc, dmin):
    maxh, maxk, maxl = uc.max_miller_indices(dmin)
    indices = []
    for h in range(-maxh, maxh + 1):
        for k in range(-maxk, maxk + 1):
            for l in range(-maxl, maxl + 1):
                if h == 0 and k == 0 and l == 0:
                    continue
                if uc.d((h, k, l)) < dmin:
                    continue
        indices.append((h, k, l))
    return indices
    
uc=(5.4307,5.4307,5.4307,90.00,90.0,90.00)
dmin=1.0
generate_reflection_indices(uc=uc,dmin=dmin)


# millerArrays readMtzFile

- Read in a mtz file into a Miller array with iotbx.file_reader.
- The `$` below marks a site for editing.

```python
from iotbx.file_reader import any_file
mtz_in = any_file("${1:data}.mtz", force_type="mtz")
miller_arrays = mtz_in.file_server.miller_arrays
```

In [ ]:
from iotbx.file_reader import any_file
mtz_in = any_file("3nd4.mtz", force_type="mtz")
miller_arrays = mtz_in.file_server.miller_arrays

# millerArrays normalizedStructureFactors

- Calculate quasi-normalized structure factor.
- The `$` below marks a site for editing.

In [ ]:
all_e_values = miller_array.quasi_normalize_structure_factors().sort(by_value="data")

# millerArrays millerArrayTruncate

- Read mtz file into a Miller array, truncate, and print summary.
- The `$` below marks a site for editing.

```python
from iotbx import mtz
mtz_obj = mtz.object(file_name="${1:3nd4}.mtz")
miller_arrays = mtz_obj.as_miller_arrays()
miller_array_truncated = miller_arrays[0].resolution_filter(d_min=${2:2}, d_max=${3:5})
print(miller_array_truncated)
miller_array_truncated.show_summary()
```

In [ ]:
from iotbx import mtz
mtz_obj = mtz.object(file_name="3nd4.mtz")
miller_arrays = mtz_obj.as_miller_arrays()
miller_array_truncated = miller_arrays[0].resolution_filter(d_min=2, d_max=5)
print(miller_array_truncated)
miller_array_truncated.show_summary()

# millerArrays 

- Read a mtz file into a miller array.
- The `$` below marks a site for editing.

```python
from iotbx.reflection_file_reader import any_reflection_file
hkl_file = any_reflection_file("${1:3hz7.mtz}")
miller_arrays = hkl_file.as_miller_arrays(merge_equivalents=False)
```

In [ ]:
from iotbx.reflection_file_reader import any_reflection_file
hkl_file = any_reflection_file("3hz7.mtz")
miller_arrays = hkl_file.as_miller_arrays(merge_equivalents=False)

# millerArrays millerArrayLabels

- Print column labels in a Miller array.

In [ ]:
[print("Miller Array %s: %s" % (i, miller_array.info().labels)) for i, miller_array in list(enumerate(miller_arrays))[:2]]

# millerArrays millerArrayWavelength

- Print wavelengths of each miller array.

In [ ]:
[print("Miller Array %s: %s" % (i, miller_array.info().wavelength)) for i, miller_array in list(enumerate(miller_arrays))]

# millerArrays millerArraySource

- Print the source of each miller array.

In [ ]:
print("Miller Array %s: %s" % (i, miller_array.info().source)) for i, miller_array in list(enumerate(miller_arrays))]

# millerArrays millerArrayLength

- Print length of miller arrays (i.e., the number of datasets in a mtz file).

In [ ]:
len(miller_arrays)

# millerArrays millerArraySymmetry

- Print the crystal symmetry of each miller array.

In [ ]:
[print("Miller Array %s: %s" % (i, miller_array.info().crystal_symmetry_from_file)) for i, miller_array in list(enumerate(miller_arrays))]

# millerArrays millerArrayHKLs

- Print all of the miller indices for a given Miller array.

In [ ]:
[print(hkl) for hkl in miller_arrays[0].indices()]

# millerArrays 

- Print the available methods for the Miller class.
- The `$` below marks a site for editing.

```python
dir(miller_arrays[${1:0}])
```


In [ ]:
dir(miller_arrays[0])

# millerArrays millerArrayDstar

- Return the resolution range in d* in a specified Miller array.
- The `$` below marks a site for editing.

```python
miller_arrays[${1:0}].min_max_d_star_sq()
```

In [ ]:
miller_arrays[0].min_max_d_star_sq()

# millerArrays millerArrayDminDmax

- Return the resolution range in Angstroms for a Miller array.
- The `$` below marks a site for editing.


```python
miller_arrays[${1:0}].d_max_min()
```

In [ ]:
miller_arrays[0].d_max_min()

# millerArrays millerArrayIoverSig

- Return the I/sig overall for a given Miller array.
- The `$` below marks a site for editing.


```python
miller_arrays[${1:0}].i_over_sig_i()
```

In [ ]:
miller_arrays[0].i_over_sig_i()

# millerArrays millerArrayIoverSig

- Return CC one-half sigma tau for a given Miller array.
- The `$` below marks a site for editing.


```python
miller_arrays[${1:0}].cc_one_half_sigma_tau()
```

In [ ]:
miller_arrays[0].cc_one_half_sigma_tau()